In [ ]:
base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=50)),
                 ('rf_2', SVC())             
                ]

clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())

# Extract score
clf.fit(spectra_train, labels_train_shape_as_array_wo_OHE).score(spectra_test, y_test)

In [ ]:
lm_size_regression = MultiOutputRegressor(LinearRegression(n_jobs = -1))

In [ ]:
labels_test_size = labels_test.drop(columns = ['index', 'Geometry_sphere', 'Geometry_wire', 'Geometry_TriangPrismIsosc', 
                                            'Geometry_parallelepiped', 'index', 'Material_Au', 'Material_SiN', 'Material_SiO2'])
labels_train_size = labels_train.drop(columns = ['index', 'Geometry_sphere', 'Geometry_wire', 'Geometry_TriangPrismIsosc', 
                                            'Geometry_parallelepiped', 'index', 'Material_Au', 'Material_SiN', 'Material_SiO2'])

labels_train_size_as_array = labels_train_size.to_numpy()
labels_test_size_as_array = labels_test_size.to_numpy()

labels_test_shape = labels_test.drop(columns = [ 'index', 'Material_Au', 'Material_SiN', 'Material_SiO2', 
                                               'log Area/Vol','ShortestDim', 'MiddleDim','LongDim'])
labels_train_shape = labels_train.drop(columns = ['index', 'Material_Au', 'Material_SiN', 'Material_SiO2', 
                                               'log Area/Vol','ShortestDim', 'MiddleDim','LongDim'])

labels_train_shape_as_array = labels_train_shape.to_numpy()
labels_test_shape_as_array = labels_test_shape.to_numpy()

labels_train_shape_as_array_wo_OHE = convert_from_one_hot(labels_train_shape_as_array, from_one_hot_dict)
labels_test_shape_as_array_wo_OHE = convert_from_one_hot(labels_test_shape_as_array, from_one_hot_dict)

In [ ]:
lm_size_regression.fit(spectra_train, labels_train_size_as_array)

In [ ]:
lm_size_regression.score(spectra_test, labels_test_size_as_array)

In [ ]:
lm_size_regression.predict(spectra_test)

In [ ]:
rf_shape_from_size = RandomForestClassifier(n_estimators = 50)

In [ ]:
rf_shape_from_size.fit(labels_train_size_as_array, labels_train_shape_as_array_wo_OHE)

In [ ]:
rf_shape_from_size.score(labels_test_size_as_array, labels_test_shape_as_array_wo_OHE)

In [ ]:
rf_size_regression = joblib.load("RF size regression.joblib")

In [ ]:
predictions = rf_size_regression[1].predict(spectra_test)

In [ ]:
shape_predictions = rf_shape_from_size.predict(predictions)

In [ ]:
cm = confusion_matrix(labels_test_shape_as_array_wo_OHE, shape_predictions)
normalized_cm = normalize_cm(cm, list(labels_test_shape_as_array_wo_OHE), 4)

In [ ]:
plot_confusion_matrix(np.asarray(normalized_cm), ["Parallelepiped", "Triangular Prism", "Sphere", "Wire"])

In [ ]:
labels_train_shape = labels_train.drop(columns = ['index','log Area/Vol', 'ShortestDim', 'MiddleDim', 'LongDim', 
                                                        'Material_Au', 'Material_SiN', 'Material_SiO2', 'index'] )
labels_test_shape = labels_test.drop(columns = ['index','log Area/Vol', 'ShortestDim', 'MiddleDim', 'LongDim', 
                                                        'Material_Au','Material_SiN', 'Material_SiO2', 'index'] )

labels_train_shape_as_array = labels_train_shape.to_numpy()
labels_test_shape_as_array = labels_test_shape.to_numpy()
        
labels_train_shape_as_array_wo_OHE = convert_from_one_hot(labels_train_shape_as_array, from_one_hot_dict)
labels_test_shape_as_array_wo_OHE = convert_from_one_hot(labels_test_shape_as_array, from_one_hot_dict)

In [ ]:
correct_prediction_indicies = []
misclassification_indicies = []
for i in range(0, len(rf_shape_classifier_predictions)):
    if rf_shape_classifier_predictions[i] == rf_shape_classifier_test_set[i]:
        correct_prediction_indicies.append(i)
    if rf_shape_classifier_predictions[i] != rf_shape_classifier_test_set[i]:
        misclassification_indicies.append(i)

In [ ]:
top_half = list(rf_shape_classifier_test_set[0:5000])
print("top half", top_half.count(0),
top_half.count(1),
top_half.count(2),
top_half.count(3))


bottom_half = list(rf_shape_classifier_test_set[5000:7811])
print("bottom half", bottom_half.count(0),
bottom_half.count(1),
bottom_half.count(2),
bottom_half.count(3))

In [ ]:
print(len(misclassification_indicies))
print(len(correct_prediction_indicies))

In [ ]:
misclassification_column = []
for i in range(0, len(misclassification_indicies) + len(correct_prediction_indicies)):
    if i in misclassification_indicies:
        misclassification_column.append(1)
    if i in correct_prediction_indicies:
        misclassification_column.append(0)

In [ ]:
len(misclassification_column)

In [ ]:
labels_test["Misclassification"] = misclassification_column

In [ ]:
misclassification_correlation_area_vol = labels_test.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "ShortestDim","MiddleDim","LongDim"])

In [ ]:
misclassification_correlation_area_vol.corr()

In [ ]:
misclassification_correlation_short_dim = labels_test.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "log Area/Vol","MiddleDim","LongDim"])

In [ ]:
misclassification_correlation_short_dim.corr()

In [ ]:
misclassification_correlation_mid_dim = labels_test.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "log Area/Vol","ShortestDim","LongDim"])
misclassification_correlation_mid_dim.corr()

In [ ]:
misclassification_correlation_long_dim = labels_test.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "log Area/Vol","ShortestDim","MiddleDim"])
misclassification_correlation_long_dim.corr()

In [ ]:
labels_test_parallelepiped = drop_indicies(labels_test, 'Geometry_parallelepiped', 0, False)[0]
labels_test_parallelepiped = labels_test_parallelepiped.reset_index()
labels_test_parallelepiped.drop(columns = "index", inplace = True)

In [ ]:
misclassification_parallelepiped_area_vol = labels_test_parallelepiped.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "ShortestDim","MiddleDim","LongDim"])
misclassification_parallelepiped_area_vol.corr()

In [ ]:
misclassification_parallelepiped_area_vol = labels_test_parallelepiped.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "ShortestDim","MiddleDim","LongDim"])
misclassification_parallelepiped_area_vol.corr()

In [ ]:
misclassification_parallelepiped_small_dim = labels_test_parallelepiped.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "log Area/Vol","MiddleDim","LongDim"])
misclassification_parallelepiped_small_dim.corr()

In [ ]:
misclassification_parallelepiped_medium_dim = labels_test_parallelepiped.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "log Area/Vol","ShortestDim","LongDim"])
misclassification_parallelepiped_medium_dim.corr()

In [ ]:
misclassification_parallelepiped_long_dim = labels_test_parallelepiped.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped", 
                                                                     "Geometry_sphere", "Geometry_wire", 
                            "Material_Au","Material_SiN","Material_SiO2", "log Area/Vol","ShortestDim","MiddleDim"])
misclassification_parallelepiped_long_dim.corr()

Build a random forest model to predict whether or not the spectrum will have its shape misclassified and examine which wavelengths are most important to determining misclasification using feature_importance_ 

use the rf model to predict shapes on spectra_train_smaller (which is all maxwell equation data) and determine which were misclassified. Build a DF with these misclassifications and use it to train a model predicting misclassification from spectrum. Test this model on labels_test to see how accurately we can predict misclassifications based on spectrum, and which wavelengths are ambigous for classification tasks

In [ ]:
top_half_spectra_test = spectra_test[0:4000]

In [ ]:
predictions = rf_shape_classifier_all.predict(spectra_test)
labels_test_misclassification = rf_shape_classifier_test_set

In [ ]:
correct_prediction_indicies = []
misclassification_indicies = []
for i in range(0, len(predictions)):
    if predictions[i] == labels_test_misclassification[i]:
        correct_prediction_indicies.append(i)
    if predictions[i] != labels_test_misclassification[i]:
        misclassification_indicies.append(i)
misclassification_column = []
for i in range(0, len(misclassification_indicies) + len(correct_prediction_indicies)):
    if i in misclassification_indicies:
        misclassification_column.append(1)
    if i in correct_prediction_indicies:
        misclassification_column.append(0)

In [ ]:
len(misclassification_column)

In [ ]:
print(sum(  misclassification_column[6000:7812]))

print(1 - 506/1812)

In [ ]:
rf_misclassification = RandomForestClassifier(n_estimators = 50, max_features = 200, n_jobs = -1, oob_score = True,
                                             warm_start = True)
rf_misclassification.fit(spectra_test[0:6000], misclassification_column[0:6000])

In [ ]:
rf_misclassification.score(spectra_test[6000:7812], misclassification_column[6000:7812])

In [ ]:
plt.plot(rf_misclassification.feature_importances_)
plt.title("Predict Misclassification From Spectrum")

In [ ]:
predictions = rf_misclassification.predict(spectra_test[6000:7812])
cm_rf = confusion_matrix(misclassification_column[6000:7812], predictions)

In [ ]:
cm_normalized = normalize_cm(cm_rf, list(misclassification_column[6000:7812]), 2)
plot_confusion_matrix(np.asarray(cm_normalized), ["Correctly Classified", "Misclasified"])

In [ ]:
rf_misclassification_from_size = RandomForestClassifier(n_estimators = 100, n_jobs = -1, oob_score = True)
rf_misclassification_from_size.fit(np.asarray(labels_test_no_misclassification)[0:6000], misclassification_column[0:6000])
print(rf_misclassification_from_size.score(np.asarray(labels_test_no_misclassification)[6000:7812], misclassification_column[6000:7812]))
predictions = rf_misclassification_from_size.predict(np.asarray(labels_test_no_misclassification)[6000:7812])
cm_rf = confusion_matrix(misclassification_column[6000:7812], predictions)
cm_normalized = normalize_cm(cm_rf, list(misclassification_column[6000:7812]), 2)
plot_confusion_matrix(np.asarray(cm_normalized), ["Correctly Classified", "Misclasified"])

In [ ]:
rf_misclassification_no_size = RandomForestClassifier(n_estimators = 100, n_jobs = -1, oob_score = True)
rf_misclassification_no_size.fit(np.asarray(labels_test_no_size)[0:6000], misclassification_column[0:6000])
print(rf_misclassification_no_size.score(np.asarray(labels_test_no_size)[6000:7812], misclassification_column[6000:7812]))
predictions = rf_misclassification_no_size.predict(np.asarray(labels_test_no_size)[6000:7812])
cm_rf = confusion_matrix(misclassification_column[6000:7812], predictions)
cm_normalized = normalize_cm(cm_rf, list(misclassification_column[6000:7812]), 2)
plot_confusion_matrix(np.asarray(cm_normalized), ["Correctly Classified", "Misclasified"])

In [ ]:
labels_test_no_misclassification =labels_test.drop(columns = ["Misclassification"])
labels_test_no_size = labels_test.drop(columns = ["Misclassification", "Geometry_TriangPrismIsosc","Geometry_parallelepiped",
                                                  "Geometry_wire", "Geometry_sphere"])
labels_test_no_size

In [ ]:
categories = ["Tri","Para", "Sphere","Wire","Au", "SiN",
          "SiO2", "A/V","Short","Mid", "Long"]

title = "Predict Misclassification From Size, Shape, and Material"

sns.barplot(categories, rf_misclassification_from_size.feature_importances_).set(title = title, ylabel = "Importance")

In [ ]:
labels_test.drop(columns = ["Geometry_TriangPrismIsosc","Geometry_parallelepiped","Geometry_sphere","Geometry_wire", "index"], inplace = True)

In [ ]:
labels_test["Shape"] = rf_shape_classifier_test_set
labels_test

In [ ]:
labels_test_material = labels_test.drop(columns = ["log Area/Vol", "ShortestDim", "MiddleDim", "LongDim", "Shape"])

In [ ]:
from_one_hot_dict_materials = {(1.,0.,0.) : 0, (0.,1.,0.) : 1, (0.,0.,1.) : 2}

In [ ]:
array_labels_test_material = convert_from_one_hot(np.asarray(labels_test_material), from_one_hot_dict_materials)

In [ ]:
array_labels_test_material

In [ ]:
labels_test["Material"] = array_labels_test_material

In [ ]:
labels_test_no_OHE = labels_test.drop(columns = ["Material_Au","Material_SiN","Material_SiO2"])